In [1]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np

from tensorflow.keras.utils import Sequence

In [2]:
MAXSEQ = 500
NUM_FILTER = 512
WINDOW_SIZES = [2,8]
DROPOUT = 0.7
NUM_HIDDEN = 500
BATCH_SIZE  = 256
NUM_CLASSES = 2
NUM_FEATURE = 1024
EPOCHS      = 20
DATA_TYPE = "ProtTrans"

In [3]:
import os

# change to your root repository
repo_root = "/mnt/D/jupyter/peter/github/DeepCytokine"
os.chdir(repo_root)
# Get the current working directory
CURDIR = os.getcwd()

print("Current Working Directory:", CURDIR)

Current Working Directory: /mnt/D/jupyter/peter/github/DeepCytokine


In [3]:
class DeepScan(Model):

	def __init__(self,
	             input_shape=(1, MAXSEQ, NUM_FEATURE),
	             window_sizes=[4,8,16],
	             num_filters=256,
	             num_hidden=1000):
		super(DeepScan, self).__init__()
		# Add input layer
		self.input_layer = tf.keras.Input(input_shape)
		self.window_sizes = window_sizes
		self.conv2d = []
		self.maxpool = []
		self.flatten = []
		for window_size in self.window_sizes:
			self.conv2d.append(
                layers.Conv2D(filters=num_filters,
    			              kernel_size=(1, window_size),
			                  activation=tf.nn.relu,
			                  padding='valid',
			                  bias_initializer=tf.constant_initializer(0.1),
			                  kernel_initializer=tf.keras.initializers.GlorotUniform()
                             )
                             )
			self.maxpool.append(
    			layers.MaxPooling2D(pool_size=(1, MAXSEQ - window_size + 1),
			                        strides=(1, MAXSEQ),
			                        padding='valid')
                               )
			self.flatten.append(layers.Flatten())
		self.dropout = layers.Dropout(rate=0.7)
		self.fc1 = layers.Dense(num_hidden,
                        		activation=tf.nn.relu,
		                        bias_initializer=tf.constant_initializer(0.1),
		                        kernel_initializer=tf.keras.initializers.GlorotUniform()
                               )
		self.fc2 = layers.Dense(NUM_CLASSES,
		                        activation='softmax',
		                        kernel_regularizer=tf.keras.regularizers.l2(1e-3)
                               )

		# Get output layer with `call` method
		self.out = self.call(self.input_layer)

	def call(self, x, training=False):
		_x = []
		for i in range(len(self.window_sizes)):
			x_conv = self.conv2d[i](x)
			x_maxp = self.maxpool[i](x_conv)
			x_flat = self.flatten[i](x_maxp)
			_x.append(x_flat)

		x = tf.concat(_x, 1)
		x = self.dropout(x, training=training)
		x = self.fc1(x)
		x = self.fc2(x)  #Best Threshold
		return x

In [4]:
path_to_val = "C:/jupyter/globe/deepcytokine/dataset/numpy/validation.npy"
print(path_to_val)
print("Validation set for DeepCytokine model: ", np.load(path_to_val).shape)

C:/jupyter/globe/deepcytokine/dataset/numpy/validation.npy
Validation set for DeepCytokine model:  (3, 1, 500, 1024)


In [5]:
# input_shape=(3, 1, 500, 1024)
model = DeepScan(input_shape=(1, MAXSEQ, NUM_FEATURE), 
                 window_sizes=WINDOW_SIZES, 
                 num_filters=NUM_FILTER, 
                 num_hidden=NUM_HIDDEN)


dummy_input = tf.random.normal([10, 1, 500, 1024])
output = model(dummy_input)  # 确保模型结构与训练时一致
print(output.shape)
# 加载训练好的权重
path_to_saved_model = "C:/jupyter/globe/deepcytokine/saved_model/latest_DeepCytokine_model.h5"
model.load_weights(path_to_saved_model)
print("weight loading complete!")

(10, 2)
weight loading complete!


In [6]:
model.summary()

Model: "deep_scan"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 499, 512)       1049088   
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 493, 512)       4194816   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 512)        0         
 )                                                               
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 512)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 flatten_1 (Flatten)         (None, 512)               0 

In [7]:
val_set = np.load(path_to_val)
model.predict(val_set)

1/1 [==============================] - 0s 500ms/step


array([[2.6116491e-04, 9.9973887e-01],
       [8.1399153e-04, 9.9918598e-01],
       [6.9800639e-01, 3.0199364e-01]], dtype=float32)